In [5]:
import numpy as np 
from numpy import cos, sin
import matplotlib.pyplot as plt
import multiprocessing
import time
# import visualisation

%matplotlib inline

import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torch.optim as optim
torch.backends.cudnn.benchmark = True 

import torchvision as tv


print("Using torch", torch.__version__)

Using torch 1.11.0


In [6]:
# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:1")
print("Using device", device)


DATASET_PATH = "ass1_data/Data/"

Using device cuda:1


In [7]:
colors = [[31, 120, 180], [51, 160, 44]]
colors = [(r / 255, g / 255, b / 255) for (r, g, b) in colors]

def plot_epochs(train_history, val_history, plotType="loss"):
    x = np.arange(1, len(train_history) + 1)
    plt.figure(figsize=(8, 6))
    plt.plot(x, val_history, color=colors[1], label=f"Validation {plotType}", linewidth=2)
    plt.plot(x, train_history, color=colors[0], label=f"Training {plotType}", linewidth=2)
    plt.ylabel(f'{plotType}')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.title(f"Evolution of the training and validation {plotType}")
    plt.show()

In [8]:
def get_cnn_output_size(in_size, kernel_size, stride=1, padding=0):
    integer = int((in_size - kernel_size + 2 * padding) / stride) == ((in_size - kernel_size + 2 * padding) / stride)
    if not integer:
        print("not int", ((in_size - kernel_size + 2 * padding) / stride))
    return int((in_size - kernel_size + 2 * padding) / stride) + 1

In [9]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():  # GPU operation have separate seed
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


set_seed(42)


num_workers = multiprocessing.cpu_count()

def gen_transforms(resize=None, imgs_mean=(0.5,), imgs_std=(0.5,)):
    tst_trans = [tv.transforms.ToTensor()]
    if resize is not None:
        tst_trans.append(tv.transforms.Resize(resize[0]))
        tst_trans.append(tv.transforms.CenterCrop(resize[1]))
    tst_trans.append(tv.transforms.Normalize(mean=imgs_mean, std=imgs_std))
    return tv.transforms.Compose(tst_trans)

def get_train_test_data(image_resize=None):
    transforms = gen_transforms(image_resize)
    trainData = tv.datasets.DatasetFolder(
        root=f'{DATASET_PATH}/training',
        transform=transforms,
        extensions=("rgb.png"),
        loader=tv.datasets.folder.default_loader)
    # a = ({
    #     loader: Callable[[str], Any],
    #     target_transform: Optional[Callable] = None,
    #     is_valid_file: Optional[Callable[[str], bool]] = None,
    # })
    return trainData


def get_train_loader(batchSize, resize=None):
    trainData = get_train_test_data(resize)
    trainLoader = td.DataLoader(trainData, batch_size=batchSize, shuffle=True, drop_last=False, num_workers=4)
    return trainLoader

trainLoader = get_train_loader(4)

In [10]:
def getLoss():
    return nn.MSELoss()

def getOptimiser(model, learningRate):
    #optimiser = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    optimiser = optim.Adam(model.parameters(), lr=learningRate)
    return optimiser

In [11]:

def train(model, batch_size, n_epochs, learning_rate, resize=None):
    print(f"batch size: {batch_size}")
    print(f"n epochs: {n_epochs}")
    print(f"learning rate: {learning_rate}")
    print("="*20)
    
    trainLoader = get_train_loader(batch_size, resize)
    lossFn = getLoss()
    optimizer = getOptimiser(model, learning_rate)

    train_history = []
    training_start_time = time.time()
    
    model = model.to(device) # Move model to gpu
    model.train() # Set the model to training mode (for DropOut)

    for epoch in range(n_epochs):  # loop over the dataset for n_epoch
        epoch_loss = 0
        
        with tqdm(trainLoader, unit="batch") as tepoch:
            for inputs, targets in tepoch:   #for each batch
                tepoch.set_description(f"Epoch {epoch}")
                # Move tensors to gpu
                inputs, targets = inputs.to(device), targets.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs)
                # _, predicted = outputs.argmax(dim=1, keepdim=True).squeeze()
                #  = torch.max(outputs, 1)
                print('out', outputs.size(), 'targets', targets.size())
                loss = lossFn(outputs, targets)
                loss.backward()
                optimizer.step()
                # add to running totals
                epoch_loss += loss.item()
        epoch_loss = epoch_loss / len(trainLoader)
        # print(f"Epoch #{epoch + 1} train_loss: {round(epoch_loss,2)} accuracy: {round(epoch_accuracy,1)}% took: {round(time.time() - start_time,2)}s") 
        train_history.append(epoch_loss)

    print(f"Training Finished, took {round(time.time() - training_start_time,1)}s")
    return train_history

In [12]:
get_cnn_output_size(259, kernel_size=3, stride=1, padding=0)

257

In [13]:
class DeepConvNN(nn.Module):
    def __init__(self):
        super(DeepConvNN, self).__init__()
        #in = 1047 x 1047
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0), #out = 523x523
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.Dropout(0.3),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0), #out = 521x521
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # nn.Dropout(0.3),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0), #out = 519x519
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0) #out = 259x259                 
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1), #out = 259x259
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # nn.Dropout(0.3),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0), #out = 129x129
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), #out = 129x129
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
            # MaxPool could be used instead of stride 2
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), #out = 127x127
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # nn.Dropout(0.3),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=0), #out = 63x63
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # nn.Dropout(0.3),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0),  #out = 61x61
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0), #out = 59x59
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # nn.Dropout(0.3),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=0), #out = 29x29
            nn.ReLU(),
            nn.BatchNorm2d(256),
            # nn.Dropout(0.3),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=0), #out = 14x14
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.flattened_size = 256 * 7 * 7 #36864

        self.regressionBlock = nn.Sequential(
            # nn.Dropout(0.5),
            nn.Linear(self.flattened_size, 512),
            nn.Linear(512, 5),
            # nn.Dropout(0.5), 
            # nn.Linear(2048, 5)  # We also found two fully connected layers to reduce accuracy
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool(x)
        x = x.view(-1, self.flattened_size)
        x = self.regressionBlock(x)
        print(x.size())
        return x

In [26]:
BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 0.01
resize = (1047, 1047)

nnet = DeepConvNN()

trains_loss = train(nnet, BATCH_SIZE, EPOCHS, LEARNING_RATE, resize)

batch size: 4
n epochs: 5
learning rate: 0.01


  0%|          | 0/8 [00:00<?, ?batch/s]

torch.Size([4, 5])
out torch.Size([4, 5]) targets torch.Size([4])


/users/sgmsempl/.conda/envs/torch/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (4) at non-singleton dimension 1